In [14]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import numpy as np
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time
#To do Regex
import re

# Anders Twitter keys
## Needs to be switched with Yuri's when we do the real pull

API Key: l8K8SocntIKK1kFE9rUDJfwO8

API Secret: 36cWKxQ59gWK7twlKTXRBqOMA4DIv16zO0pjlBHgfxH4szCQAG

Bearer Token: AAAAAAAAAAAAAAAAAAAAAHWGfwEAAAAATNqL7vFiLYOENaicSGgIuJbnejI%3DFNW4rnpK7FpmBDZrCfNpsdMaVKvo5HKXHedygV4r7dJSbbgsSC

# Defining the necessary functions

In [15]:
#Define Bearer token in the Windows OS Enviroment (CHANGE TOKEN HERE)
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAHWGfwEAAAAATNqL7vFiLYOENaicSGgIuJbnejI%3DFNW4rnpK7FpmBDZrCfNpsdMaVKvo5HKXHedygV4r7dJSbbgsSC'

#Defining function that will fetch the token from the enviroment
def auth():
    return os.getenv('TOKEN')

#Defining function that will pass the token for authorization and return headers for use w. the API.
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

#Defining function that creates the URL for the correct endpoint. ()
def create_url(keyword, start_date, end_date, max_results):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

# The function below will send the “GET” request and if everything is correct 
# (response code 200), it will return the response in “JSON” format.

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

# Create request

In the next cell, we will set up our inputs:

    keyword = \"doge -is:retweet lang:en\"
The above specification looks for keyword "doge", exclude any retweets, and search for english tweets only bearer_token and headers from the API.

### First - define inputs

In [16]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "doge -is:retweet lang:en"
start_time = "2021-03-01T00:00:00.000Z"
end_time = "2021-03-31T00:00:00.000Z"
max_results = 15

### Test field (Pulls only 10 Tweets)

In [17]:
url = create_url(keyword, start_time, end_time, max_results)

json_response = connect_to_endpoint(url[0], headers, url[1])

df = pd.DataFrame(json_response['data'])

df
#df['test']

Endpoint Response Code: 403


Exception: (403, '{"client_id":"25134709","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Standard Basic","reason":"client-not-enrolled","type":"https://api.twitter.com/2/problems/client-forbidden"}')

# Full pull

### Define function to loop the pull several times. Saves as DataFrame.

In [153]:
def twitter_pull(maxreps=4):
    
    #Setup
    reps=0
    next_token=None
    df_result = pd.DataFrame()
    
    #Looping part
    
    for i in range(maxreps):
        #Get data from Twitter API
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        
        #Check for next_token being fetched successfully.
        #If yes, print it and continue
        if 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
            print(f'next_token: {next_token}')
        #If no, print it and continue
        else:
            print('next token error')
            break
        
        df_response = pd.DataFrame(json_response['data'])
        df_result = df_result.append(df_response, ignore_index=True)
        reps+=1
        time.sleep(2)
        print(f'reps completed: {reps}')
    
    print(f'Number of tweets pulled: {len(df_result)}')
    
    return df_result
    
    

# Actual pull field:

If maxreps=4 is used, then 400 Tweets will be pulled.

In [154]:
testest = twitter_pull(4)

testest

Endpoint Response Code: 200
next_token: b26v89c19zqg8o3fpz5nhbci7xqqhf9r0cmmzkc2nc8sd
reps completed: 1
Endpoint Response Code: 200
next_token: b26v89c19zqg8o3fpz5nhbci7x3nt71ylkuhatx91ol8d
reps completed: 2
Endpoint Response Code: 200
next_token: b26v89c19zqg8o3fpz5nhbci7x3f5yfbcpvzs53tj1px9
reps completed: 3
Endpoint Response Code: 200
next_token: b26v89c19zqg8o3fpz5nhbci6g6kv9yts91x2m99yux6l
reps completed: 4
Number of tweets pulled: 40


In [155]:
testest

,conversation_id,public_metrics,text,author_id,created_at,in_reply_to_user_id,lang,reply_settings,referenced_tweets,id,source,geo
0,1557876793050181633,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","@TheCryptoLark Hodl bitcoin, bnb, doge, sol, e...",621139488,2022-08-12T13:35:21.000Z,30325257,en,everyone,"[{'type': 'replied_to', 'id': '155787679305018...",1558084725503594499,Twitter Web App,NaN
1,1558084710303154176,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","The current price of one DOGE is now: $0.0706,...",828615194771353600,2022-08-12T13:35:17.000Z,NaN,en,everyone,NaN,1558084710303154176,IFTTT,NaN
2,1558084684638310402,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Everyone trusted $doge $shiba $luna some prove...,1505950409747320838,2022-08-12T13:35:11.000Z,NaN,en,everyone,"[{'type': 'quoted', 'id': '1558082364189954050'}]",1558084684638310402,Twitter for Android,NaN
3,1557635423127670788,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@KuCoin_FR @amamira899 @kucoincom @Archie9635 ...,1434094229161013254,2022-08-12T13:35:03.000Z,1191665457725661186,en,everyone,"[{'type': 'replied_to', 'id': '155763542312767...",1558084653915193344,Twitter for Android,NaN
4,1534087379601924096,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@MetascopeNFT 🔥Miss the SHIB or DOGE? Finding ...,1535637593236221952,2022-08-12T13:34:52.000Z,1489070082198106113,en,everyone,"[{'type': 'replied_to', 'id': '153408737960192...",1558084604640333825,Twitter Web App,NaN
5,1557943469984956417,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",@elonmusk Doge 69 $$$,1507992243772489744,2022-08-12T13:34:49.000Z,44196397,en,everyone,"[{'type': 'replied_to', 'id': '155794346998495...",1558084592023871494,Twitter for Android,NaN
6,1558030303481970688,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@jonboyjon1976 @doge_techno5 LOL 😆 That's some...,1500782287037808642,2022-08-12T13:34:42.000Z,66423339,en,everyone,"[{'type': 'replied_to', 'id': '155808244683871...",1558084563095764999,Twitter for Android,NaN
7,1558084556120801281,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",$DOGE Earnings reports today before the market...,1530945555668422659,2022-08-12T13:34:40.000Z,NaN,en,everyone,NaN,1558084556120801281,Twitter for Android,NaN
8,1558084499401019393,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@elonmusk DALL-E should accept Doge\nSuch art\...,14459572,2022-08-12T13:34:27.000Z,44196397,en,everyone,NaN,1558084499401019393,Twitter Web App,NaN
9,1558084497542979586,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","However many holy words you read, however many...",1488892394548469765,2022-08-12T13:34:26.000Z,NaN,en,everyone,NaN,1558084497542979586,OwnerOfGMT,NaN
